In [ ]:
import re
import json

# 读取txt文件
with open('attack copy.log') as f:
    lines = f.readlines()

cluster_calls = dict()

for line in lines:
    cluster = eval(line.split('action in cluster ')[1][0])
    api_call = line.split('action in cluster ')[1].split(':')[1].strip().split(' ')[0]
    if cluster not in cluster_calls:
        cluster_calls[cluster] = []
    if api_call not in cluster_calls[cluster]:
        cluster_calls[cluster].append(api_call)

# 输出到json文件    
with open('effi_calls.json', 'w') as f:
  json.dump(cluster_calls, f, indent=4)

In [ ]:
import random
for i in range(10):
    print(random.uniform(0, 1))

In [ ]:
import os

def extract_class_name(string):
    return '/'.join(string.split('(')[0].split(".")[0:-1])

def get_smali(action):
    # action对应的apk的解析位置
    tmp_dir = f"/home/hlj/dev/shm/gnip/smali/{os.path.basename(action.apk_path.strip()).replace('.apk','')}/"
    if not os.path.exists(tmp_dir):
        cmd = f"apktool d {action.apk_path.strip()} -o {tmp_dir} -r -f"
        os.system(cmd)
    smali_path = f"{tmp_dir}/smali/{extract_class_name(action.caller)}.smali"
    i=1
    while not os.path.exists(smali_path):
        i+=1
        smali_path = f"{tmp_dir}/smali_classes{i}/{extract_class_name(action.caller)}.smali"
    # 打开对应smali文件，找到相应method
    with open(smali_path, 'r') as f:
        smali_content = f.readlines()
    if not os.path.exists(f"/data/hlj/datas/cic20/code/{action.cluster}/"):
        os.makedirs(f"/data/hlj/datas/cic20/code/{action.cluster}/")
    with open(f"/data/hlj/datas/cic20/code/{action.cluster}/{action.caller}.txt",'w') as file:
        file.writelines(smali_content)
        # smali_content = smali_content[3:]
    return smali_content

def load_smali(action):
    with open(action.smali_apth,'r') as file:
        return file.readlines()

def insert_smali_imp(parse_dir,ori_caller,action):
    smali_content = ""
    if action.smali != None:
        smali_content = action.smali
    elif os.path.exists(action.smali_path):
        smali_content = load_smali(action)
    else:
        smali_content = get_smali(action)
        # smali_content = get_code(apk_path,action.caller,action.cluster)
        action.smali = smali_content

    # smali_content = get_smali(apk_path,action.caller)

    dir = '/'.join(action.caller.split('(')[0].split('.')[:-2])
    name = action.caller.split('(')[0].split('.')[-2]
    new_api = action.caller
    new_smali_path = f'{parse_dir}/smali/{dir}/{name}.smali'
    if not os.path.exists(os.path.join(parse_dir+'/smali',dir)):
        os.makedirs(os.path.join(parse_dir+'/smali',dir),exist_ok=True)
    else:
        i=1
        new_smali_path = f'{parse_dir}/smali/{dir}/{name}${i}.smali'
        while os.path.exists(new_smali_path):
            print(i)
            i+=1 
            new_smali_path = f'{parse_dir}/smali/{dir}/{name}${i}.smali'
            new_api = f"{action.caller.split('(')[0]}${i}({action.caller.split('(')[1]}"
        smali_content[0] = smali_content[0].replace(';',f"${i};")
    # new_smali_path = f'{parse_dir}/smali/{dir}/{name}.smali'
    with open(new_smali_path,'w') as file:
        file.writelines(smali_content)
    # insert_line(parse_dir,ori_caller,new_api)

parse_dir = "/home/hlj/dev/shm/gnip/tmp/tmpjj2o351y/"
ori_caller = "Lcom.android.market.AutoRun.onReceive(android.content.Context,android.content.Intent)void"
insert_smali_imp()